In [2]:
# All import and installation
!pip install pyspark
import pyspark
from datetime import datetime
sc = pyspark.SparkContext('local[*]')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1c35f51b453529f046bab9fc58cc34aa0f133bc788e743b7dbc95e3188ed0e7e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [5]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://a...content-available-to-author-only...e.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
 
!ls
 
import findspark
findspark.init()
 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,049 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,240 kB]
Get:14 h

In [ ]:
#Retrive the first 10 records
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
data.take(10)

['aa 271_a.C 1 4675',
 'aa Category:User_th 1 4770',
 'aa Chiron_Elias_Krase 1 4694',
 'aa Dassault_rafaele 2 9372',
 'aa E.Desv 1 4662',
 'aa File:Wiktionary-logo-en.png 1 10752',
 'aa Indonesian_Wikipedia 1 4679',
 'aa Main_Page 5 266946',
 'aa Requests_for_new_languages/Wikipedia_Banyumasan 1 4733',
 'aa Special:Contributions/203.144.160.245 1 5812']

In [ ]:
#Q1-->max for page size

def parse(line):
  col=line.split(" ")
  pagesize=float(col[3])
  key=col[0]
  return(key,pagesize)
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pagesize=data.map(parse)
maxx=pagesize.reduceByKey(lambda x , y: max(x,y))
maxx.saveAsTextFile('max.txt')

In [ ]:
#Q1-->min for page size

def parse_linee(line):
  col=line.split(" ")
  pagesize=float(col[3])
  key=col[0]
  return (key,pagesize)
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pagesize=data.map(parse_linee)
minn=pagesize.reduceByKey(lambda x , y: min(x,y))
minn.saveAsTextFile('min.txt')

In [ ]:
#Q1-->avg for page size

def parsee(line):
  col=line.split(" ")
  page_size=float(col[3])
  return (page_size)
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
page_size=data.map(parsee)
listofavg=[]
resultt=page_size.mean()
listofavg.append(resultt)
final_rdd=sc.parallelize(listofavg)
final_rdd.saveAsTextFile('avg.txt')

In [ ]:
#Q1 --> not using min,max ,avg
# Define the parse function to extract the page size from each line
def parse(line):
    col = line.split(" ")
    pagesize = float(col[3])
    key = col[0]
    return (key, pagesize)

# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

# Start the timer for map-reduce paradigm
start_time = datetime.now()

# Map each line to a tuple
pagesize = data.map(parse)

# Reduce the tuples by key to find the maximum and minimum page sizes
max_size = pagesize.reduceByKey(lambda x, y: x if x > y else y)
min_size = pagesize.reduceByKey(lambda x, y: x if x < y else y)

# Compute the average page size
total_size = pagesize.reduceByKey(lambda x, y: x + y).values().sum()
num_pages = pagesize.count()
avg_size = total_size / num_pages

end_time = datetime.now()
execution_time_map_reduce = end_time - start_time

# Save the maximum, minimum, and average page sizes to separate text files
max_size.saveAsTextFile("max_size.txt")
min_size.saveAsTextFile("min_size.txt")
sc.parallelize([avg_size]).saveAsTextFile("avg_size.txt")
print("Execution Time - Map-Reduce Paradigm:", execution_time_map_reduce)


Execution Time - Map-Reduce Paradigm: 0:00:12.833503


In [ ]:
#َQ2--> number of page title start with The and not en
def parse_data(line):
  collomns=line.split(" ")
  return(collomns[0],collomns[1])
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

# Start the timer for map-reduce paradigm
start_time = datetime.now()

page_title=data.map(parse_data)
result=page_title.filter(lambda x: (x[1].startswith('The_'))&('en'!= x[0]))
pagetitlecount=result.distinct().count()
rddd=sc.parallelize([pagetitlecount])
end_time = datetime.now()
execution_time_map_reduce = end_time - start_time
rddd.saveAsTextFile('start.txt')
print("Execution Time - Map-Reduce Paradigm:", execution_time_map_reduce)

Execution Time - Map-Reduce Paradigm: 0:00:09.418443


In [ ]:
#Q3--> number of unique terms in page title
import re
def parseLine(line):
 fields = line.split(" ")
 f1=fields[1]
 s = re.sub(r'[^a-zA-Z_]','', f1).lower()
 return s

def parseLine22(line):
 fields = line.split("_")
 return fields
data=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()

pagetitle=data.map(parseLine)# page title
page_titlev2=pagetitle.map(parseLine22)
unique_terms=page_titlev2.flatMap(lambda x:x).distinct().count()
rdd_count = sc.parallelize([unique_terms])
end_time = datetime.now()
execution_time_map_reduce = end_time - start_time
rdd_count.saveAsTextFile('unique.txt')#list RDD
print("Execution Time - Map-Reduce Paradigm:", execution_time_map_reduce)

Execution Time - Map-Reduce Paradigm: 0:00:36.622755


In [ ]:
#Q4--> title and the number of times it was repeated.
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()

# Map each line to a tuple containing the page title and a count of 1
title_counts = data.map(lambda line: (line.split(" ")[1], 1))

# Reduce the tuples by key to count the number of occurrences of each page title
title_counts = title_counts.reduceByKey(lambda x, y: x + y)
end_time = datetime.now()
execution_time_map_reduce = end_time - start_time

# Save the output to a text file
output = "\n".join([f"{title} {count}" for title, count in title_counts.collect()])
with open("repeat.txt", "w") as f:
    f.write(output)
print("Execution Time - Map-Reduce Paradigm:", execution_time_map_reduce)

Execution Time - Map-Reduce Paradigm: 0:00:00.023023


In [10]:
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()

# Map each line to a tuple containing the page title and the rest of the line
data = data.map(lambda line: (line.split(" ")[1], line))

# Group the data by page title
grouped_data = data.groupByKey()

# Combine the data of pages with the same title and print the result
result = grouped_data.flatMap(lambda x: [(x[0], [i, j]) for i in x[1] for j in x[1] if i < j])

end_time = datetime.now()
execution_time_map_reduce = end_time - start_time

result.saveAsTextFile("combine123.txt")
print("Execution Time - Map-Reduce Paradigm:", execution_time_map_reduce)

Execution Time - Map-Reduce Paradigm: 0:00:00.041940


# **Sparks Loop**

In [ ]:
#Q1
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()

# Convert each line to a tuple containing the page title and size
pages = []
for line in data.collect():
    page_title = line.split(" ")[1]
    page_size = int(line.split(" ")[3])
    pages.append((page_title, page_size))

# Compute the min, max, and average page size using loops
min_size = float('inf')
max_size = float('-inf')
total_size = 0
num_pages = 0

for page in pages:
    size = page[1]
    if size < min_size:
        min_size = size
    if size > max_size:
        max_size = size
    total_size += size
    num_pages += 1

# Compute the average page size using loops
sum_size = 0
for page in pages:
    sum_size += page[1]
avg_size = sum_size / num_pages
end_time = datetime.now()
execution_time_sparks_loops = end_time - start_time

# Save the output to a text file
output = "Min page size: {}\nMax page size: {}\nAvg page size: {}".format(min_size, max_size, avg_size)
with open("minmaxavg.txt", "w") as f:
    f.write(output)
print("Execution Time - Sparks loops:", execution_time_sparks_loops)

Execution Time - Sparks loops: 0:00:18.403614


In [ ]:
#Q2 
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()

# Initialize counters
num_the_pages = 0
num_non_en_the_pages = 0

# Loop over all the data to count the number of "The" pages and non-English "The" pages
for line in data.collect():
    fields = line.split(" ")
    if fields[1].startswith("The_"):
        num_the_pages += 1
        if not fields[0].startswith("en"):
            num_non_en_the_pages += 1
end_time = datetime.now()          
execution_time_sparks_loops = end_time - start_time

# Save the output to a text file
output = "Number of page titles that start with 'The': {}\nNumber of 'The' pages that are not part of the English project: {}".format(num_the_pages, num_non_en_the_pages)
with open("start2.txt", "w") as f:
    f.write(output)
print("Execution Time - Sparks loops:", execution_time_sparks_loops)

Execution Time - Sparks loops: 0:00:16.952353


In [13]:
#Q3 --> number of unique terms in page titles.
from pyspark import SparkContext
import re
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
start_time = datetime.now()
# Initialize a set to store unique terms
unique_terms = set()

# Loop over all the data to extract terms and add them to the set
for line in data.collect():
    # Extract the page title and normalize it
    title = line.split(" ")[1].lower()
    title = re.sub(r'[^a-zA-Z0-9_]+', '', title)  # Remove non-alphanumeric characters
    terms = title.split("_")
    # Add each term to the set of unique terms
    for term in terms:
        if term:
            unique_terms.add(term)

# Compute the number of unique terms
num_unique_terms = len(unique_terms)

end_time = datetime.now()
execution_time_sparks_loops = end_time - start_time
# Save the output to a text file
output = "Number of unique terms in page titles: {}".format(num_unique_terms)
with open("unique2.txt", "w") as f:
    f.write(output)
print("Execution Time - Sparks loops:", execution_time_sparks_loops)

Execution Time - Sparks loops: 0:00:27.003999


In [ ]:
#Q4
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = datetime.now()
# Initialize a dictionary to store page title counts
title_counts = {}

# Loop over all the data to count the number of occurrences of each page title
for line in data.collect():
    title = line.split(" ")[1]
    if title in title_counts:
        title_counts[title] += 1
    else:
        title_counts[title] = 1

end_time = datetime.now()
execution_time_sparks_loops = end_time - start_time
# Save the output to a text file
output = "\n".join([f"{title} {count}" for title, count in title_counts.items()])
with open("repeat2.txt", "w") as f:
    f.write(output)
print("Execution Time - Sparks loops:", execution_time_sparks_loops)

Execution Time - Sparks loops: 0:00:17.223693


In [ ]:
#Q5
# Load the page view statistics data
data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
start_time = datetime.now()

# Initialize a dictionary to store page data
page_data = {}

# Loop over all the data to combine the data of pages with the same title
for line in data.collect():
    # Extract the page title from the line
    title = line.split(" ")[1]
    # Add the line to the list associated with the title key in the dictionary
    if title in page_data:
        page_data[title].append(line)
    else:
        page_data[title] = [line]

# Loop over all the groups of pages with the same title to create pairs of pages to be displayed
page_pairs = []
for title, data in page_data.items():
    # Loop over all possible pairs of lines within the group
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            # Add each pair of lines to a list
            page_pairs.append((title, [data[i], data[j]]))
end_time = datetime.now()
execution_time_sparks_loops = end_time - start_time
# Save the output to a text file
with open("combine1.txt", "w") as f:
    for pair in page_pairs:
        f.write(f"{pair}\n")
print("Execution Time - Sparks loops:", execution_time_sparks_loops)

Execution Time - Sparks loops: 0:00:29.865477


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/MyFolder
!cp -r /content/* /content/drive/MyDrive/MyFolder/